In [ ]:
# pip install fast_ml

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm
from fast_ml.model_development import train_valid_test_split
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd 
import numpy as np

# datasets
powerConsumption = pd.read_csv('C:/Users/moyin/Downloads/data/Tetuan City power consumption.csv')
activity_wsdata = pd.read_csv('C:/Users/moyin/Downloads/data/activity data.csv')
ALEdata = pd.read_csv('C:/Users/moyin/Downloads/data/ALE in Sensor.csv')
Occupancy = pd.read_csv('C:/Users/moyin/Downloads/data/Occupancy.csv')
poisioned_data = pd.read_csv('C:/Users/moyin/Downloads/data/bad_dataset.csv')

# poisoned data 
target = 'Occupancy'
X_p = poisioned_data.drop(columns=[target])
y_p = poisioned_data[target]

# split data
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p, test_size=0.5, random_state=42)
model_train_p, model_test_p, target_train_p, target_test_p = train_test_split(X_train_p, y_train_p, test_size=0.5, random_state=42)
edgeDevice_X_p, edgeData_X_p, edgeDevice_Y_p, edgeData_Y_p = train_test_split(model_train_p, target_train_p, train_size=0.5, random_state=42)
edgeServer1_X_p, edgeServer2_X_p, edgeServer1_Y_p, edgeServer2_Y_p = train_test_split(edgeDevice_X_p, edgeData_X_p, train_size=0.5, random_state=42)


# clean data 
target = 'Occupancy'
X = Occupancy.drop(columns=[target])
y = Occupancy[target]

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
model_train, model_test, target_train, target_test = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
edgeDevice_X, edgeData_X, edgeDevice_Y, edgeData_Y = train_test_split(model_train, target_train, train_size=0.5, random_state=42)
edgeServer1_X, edgeServer2_X, edgeServer1_Y, edgeServer2_Y = train_test_split(edgeDevice_X, edgeData_X, train_size=0.5, random_state=42)

In [ ]:
import hashlib
import time
import random as rand
import pandas as pd
import numpy as np


# node representing each block
class Node:
    
    def __init__(self, idx, prev_hash, t_stamp, transactions, proof, validator):
        self.idx = idx
        self.prev_hash = prev_hash
        self.t_stamp = t_stamp
        self.transactions = transactions
        self.proof = proof
        self.validator = validator
        self.hash = self.calculate_hash()
    
    def calculate_hash(self):
        node_str = f"{self.idx}{self.prev_hash}{self.t_stamp}{self.transactions}{self.proof}{self.validator}"
        return hashlib.sha256(node_str.encode('utf-8')).hexdigest()
        
        
class Blockchain_Network:
    
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.c_node_idx = 0         
        self.validators = ['R_Edge_Server_A', 'R_Edge_Server_B', 'R_Edge_Server_C', 'R_Edge_Server_D']
        self.add_node()
        self.consensus_threshold = 2  
    
    def add_node(self):
        # initial block
        if self.c_node_idx == 0:
            start_node = Node(0, "0", time.time(), [], 'Start', 'System')
            self.chain.append(start_node)
            self.c_node_idx += 1
    
    # simulate a transaction creation  data: model's data, accuracy: model's accuracy
    def transaction_session(self, source, receiver, data, accuracy):
        transaction = {"AI Vendor": source, "Edge Device": receiver, "model data": data, "model accuracy": accuracy}
        self.transactions.append(transaction)
        
    # Simulate the mining process
    def mine_node(self):
        
        validator = rand.choice(self.validators) 
        t_stamp = time.time()
        
        # Simulate proof of authority by generating a random proof
        proof = rand.randint(150, 8500)  
        node = Node(self.c_node_idx, self.chain[-1].hash, t_stamp, self.transactions, proof, validator)

        # Check for data integrity and accuracy before adding the node
        integrity_check = self.data_integrity(node.transactions)
        accuracy_check = self.is_valid_model(node.transactions)

        if not integrity_check or not accuracy_check:
            if not integrity_check:
                print(f"Node {node.idx} failed the data integrity check. Node discarded and mining continues.")
            if not accuracy_check:
                print(f"Node {node.idx} failed the model accuracy check. Node discarded and mining continues.")

            self.c_node_idx += 1
            continue
        
        

        votes = self.node_votes(node)
        # print(f"Validator Votes: {votes}")
        
        if integrity_check and accuracy_check:
            # Simulate validator votes for the block
            votes = self.node_votes(node)
            
            if len(votes) >= self.consensus_threshold: 
                self.chain.append(node)  
                self.c_node_idx += 1
                self.transactions = [] 
                
                print(f"Node {node.idx} successfully added to the blockchain.")
            else:          
                print(f"Node {node.idx} failed to reach consensus. Node discarded and mining continues.")
        return node
            

     # Simulate validators voting on the block   
    def node_votes(self, node):
        votes = []
        
        for validator in self.validators:
            # if self.data_integrity(node.transactions) and self.is_valid_model(node.transactions):
                votes.append(validator)   
        return votes


    def data_integrity(self, transactions):
        # accuracy, consistency, data mismatch, and outliers 
        for transaction in transactions:
            if isinstance(transaction['model data'], pd.DataFrame):
                missing_values = transaction['model data'].isnull().sum().sum()
                duplicate_values = transaction['model data'].duplicated().sum()
                incorrect_data = 0
                
                for col in transaction['model data'].select_dtypes(include=[np.number]).columns:
                    if not pd.api.types.is_numeric_dtype(transaction['model data'][col]):
                        incorrect_data += 1
                
                if missing_values > 0 or duplicate_values > 0 or incorrect_data > 0:
                    return False
            else:
                # Assuming model data is numeric
                if not isinstance(transaction['model data'], (int, float)):
                    return False

        return True
        
        
    def is_valid_model(self, transactions):
        
        expected_accuracy = 80 
       
        for transaction in transactions:
            if 'model accuracy' in transaction:
                m_accuracy = transaction['model accuracy']
                if m_accuracy < expected_accuracy:
                    return False             
        return True  

    
    def get_last_node(self):
        return self.chain[-1]

    
    def display_chain(self):
        for node in self.chain:
            print(f"------ Session Begins --------")
            print(f"Node {node.idx}: {node.hash} | Validator: {node.validator} | Proof: {node.proof}")
            # print(f"Transactions: {node.transactions}")
            print(f"----------------------------")
            print(f"----------------------------")



In [ ]:
# Initialize the blockchain
PoA_consensus = Blockchain_Network()

# Simulate creating transactions
    # source: AI Marketplace represented as App_VendorID
    # receiver: edge devices defined as ED_RegionID_DeviceID where ED represents the edge device
    # model data: the AI model parameters
    # model accuracy: the accuracy given by the vendor

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 1
PoA_consensus.transaction_session("App_V01", "ED_R1_AR1", model_train, 85)
PoA_consensus.transaction_session("App_V01", "ED_R2_AR2", edgeDevice_X, 80)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 2
PoA_consensus.transaction_session("App_V03", "ED_R4_AR1", edgeDevice_X, 90)
PoA_consensus.transaction_session("App_V04", "ED_R3_AR2", X_train, 100)

# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 3
PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", edgeServer1_X, 82)
PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", edgeDevice_X, 81)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 4
PoA_consensus.transaction_session("App_V04", "ED_R3_AR4", edgeServer1_X, 95)
PoA_consensus.transaction_session("App_V03", "ED_R3_AR2", edgeServer1_X, 80)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 5
PoA_consensus.transaction_session("App_V02", "ED_R4_AR1", model_train, 100)
PoA_consensus.transaction_session("App_V01", "ED_R2_AR2", edgeDevice_X, 100)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 6
PoA_consensus.transaction_session("App_V01", "ED_R2_AR4", X_train, 95)
PoA_consensus.transaction_session("App_V03", "ED_R2_AR2", edgeDevice_X, 80)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 7
PoA_consensus.transaction_session("App_V02", "ED_R3_AR4", model_train, 85)
PoA_consensus.transaction_session("App_V01", "ED_R2_AR2", edgeServer2_X, 80)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 8
PoA_consensus.transaction_session("App_V04", "ED_R2_AR4", edgeServer2_X, 85)
PoA_consensus.transaction_session("App_V03", "ED_R2_AR5", edgeData_X_p, 80)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 9
PoA_consensus.transaction_session("App_V02", "ED_R4_AR1", model_train, 85)
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", edgeDevice_X, 80)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 10
PoA_consensus.transaction_session("App_V03", "ED_R1_AR1", model_train_p, 85)
PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", edgeDevice_X_p, 80)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 11
PoA_consensus.transaction_session("App_V02", "ED_R4_AR1", model_train, 85)
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", edgeDevice_X, 80)
# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()


In [ ]:
# increase the number of edge servers to 10 -15 servers; randomly assign poision the data that is passed to it
# if all the values is 100% for the TP and FP, no need for graph...


In [ ]:
import hashlib
import time
import random as rand
import pandas as pd
import numpy as np

# Node representing each block
class Node:
    def __init__(self, idx, prev_hash, t_stamp, transactions, proof, validator):
        self.idx = idx
        self.prev_hash = prev_hash
        self.t_stamp = t_stamp
        self.transactions = transactions
        self.proof = proof
        self.validator = validator
        self.hash = self.calculate_hash()
    
    def calculate_hash(self):
        node_str = f"{self.idx}{self.prev_hash}{self.t_stamp}{self.transactions}{self.proof}{self.validator}"
        return hashlib.sha256(node_str.encode('utf-8')).hexdigest()


class Blockchain_Network:
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.c_node_idx = 0         
        self.validators = ['R_Edge_Server_A', 'R_Edge_Server_B', 'R_Edge_Server_C', 'R_Edge_Server_D']
        self.add_node()
        self.consensus_threshold = 2  
    
    def add_node(self):
        # Initial block
        if self.c_node_idx == 0:
            start_node = Node(0, "0", time.time(), [], 'Start', 'System')
            self.chain.append(start_node)
            self.c_node_idx += 1
    
    # Simulate a transaction creation data: model's data, accuracy: model's accuracy
    def transaction_session(self, source, receiver, data, accuracy):
        transaction = {"AI Vendor": source, "Edge Device": receiver, "model data": data, "model accuracy": accuracy}
        self.transactions.append(transaction)
        
    # Simulate the mining process
    def mine_node(self):
        while True:
            validator = rand.choice(self.validators) 
            t_stamp = time.time()

            # Simulate proof of authority by generating a random proof
            proof = rand.randint(150, 8500)  
            node = Node(self.c_node_idx, self.chain[-1].hash, t_stamp, self.transactions, proof, validator)

            # Check for data integrity and accuracy before adding the node
            integrity_check = self.data_integrity(node.transactions)
            accuracy_check = self.is_valid_model(node.transactions)

            # If node fails integrity or accuracy check, log the failure but continue mining
            if not integrity_check or not accuracy_check:
                if not integrity_check:
                    print(f"Node {node.idx} failed the data integrity check. Skipping this node.")
                if not accuracy_check:
                    print(f"Node {node.idx} failed the model accuracy check. Skipping this node.")

                # Increment the node index and continue to try mining the next node
                self.c_node_idx += 1
                continue  # Go back to the top of the while loop and try mining a new node
            
            # Simulate validator votes for the block
            votes = self.node_votes(node)
            if len(votes) >= self.consensus_threshold:
                self.chain.append(node)  
                self.transactions = [] 
                print(f"Node {node.idx} successfully added to the blockchain.")
                break  # Exit the loop when the node is valid and added to the chain
            else:          
                print(f"Node {node.idx} failed to reach consensus. Skipping this node and continuing mining.")  
                self.c_node_idx += 1  # Increment the index after a failed consensus
                continue  # Continue mining the next node

    # Simulate validators voting on the block   
    def node_votes(self, node):
        votes = []
        
        for validator in self.validators:
            # Only cast a vote if both integrity and accuracy checks pass
            if self.data_integrity(node.transactions) and self.is_valid_model(node.transactions):
                votes.append(validator)   
        return votes


    def data_integrity(self, transactions):
        # Debugging print to see the transactions
        # print("Checking data integrity...")
        for transaction in transactions:
            # print(f"Transaction Data: {transaction}")
            
            if isinstance(transaction['model data'], pd.DataFrame):
                missing_values = transaction['model data'].isnull().sum().sum()
                duplicate_values = transaction['model data'].duplicated().sum()
                incorrect_data = 0
                
                for col in transaction['model data'].select_dtypes(include=[np.number]).columns:
                    if not pd.api.types.is_numeric_dtype(transaction['model data'][col]):
                        incorrect_data += 1
                
                if missing_values > 0 or duplicate_values > 0 or incorrect_data > 0:
                    # print(f"Integrity issue found: Missing {missing_values}, Duplicates {duplicate_values}, Incorrect Data {incorrect_data}")
                    return False
            else:
                # Assuming model data is numeric
                if not isinstance(transaction['model data'], (int, float)):
                    print(f"Integrity issue found: Invalid data type {type(transaction['model data'])}")
                    return False
        
        return True
        
        
    def is_valid_model(self, transactions):
        expected_accuracy = 80  # Minimum acceptable accuracy
       
        for transaction in transactions:
            if 'model accuracy' in transaction:
                m_accuracy = transaction['model accuracy']
                if m_accuracy < expected_accuracy:
                    print(f"Accuracy issue: Model accuracy {m_accuracy} is below threshold.")
                    return False  # Accuracy is not valid
        return True  

    
    def get_last_node(self):
        return self.chain[-1]

    
    def display_chain(self):
        for node in self.chain:
            print(f"------ Session Begins --------")
            print(f"Node {node.idx}: {node.hash} | Validator: {node.validator} | Proof: {node.proof}")
            # print(f"Transactions: {node.transactions}")
            print(f"----------------------------")
            print(f"----------------------------")


In [ ]:
import hashlib
import time
import random as rand
import pandas as pd
import numpy as np

# Node representing each block
class Node:
    def __init__(self, idx, prev_hash, t_stamp, transactions, proof, validator):
        self.idx = idx
        self.prev_hash = prev_hash
        self.t_stamp = t_stamp
        self.transactions = transactions
        self.proof = proof
        self.validator = validator
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        node_str = f"{self.idx}{self.prev_hash}{self.t_stamp}{self.transactions}{self.proof}{self.validator}"
        return hashlib.sha256(node_str.encode('utf-8')).hexdigest()

class Blockchain_Network:
    
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.c_node_idx = 0         
        self.validators = ['R_Edge_Server_A', 'R_Edge_Server_B', 'R_Edge_Server_C', 'R_Edge_Server_D']
        self.add_node()
        self.consensus_threshold = 2  

    def add_node(self):
        # initial block
        if self.c_node_idx == 0:
            start_node = Node(0, "0", time.time(), [], 'Start', 'System')
            self.chain.append(start_node)
            self.c_node_idx += 1

    # Simulate a transaction creation with model data and accuracy
    def transaction_session(self, source, receiver, data, accuracy):
        transaction = {"AI Vendor": source, "Edge Device": receiver, "model data": data, "model accuracy": accuracy}
        self.transactions.append(transaction)

        
    # Simulate the mining process
    def mine_node(self):
        
        validator = rand.choice(self.validators) 
        t_stamp = time.time()
        
        # Simulate proof of authority by generating a random proof
        proof = rand.randint(150, 8500)  
        node = Node(self.c_node_idx, self.chain[-1].hash, t_stamp, self.transactions, proof, validator)

        # Check for data integrity and accuracy before adding the node
        integrity_check = self.data_integrity(node.transactions)
        if not integrity_check:
            print(f"Node {node.idx} failed the data integrity check. Node discarded.")
            return None

        accuracy_check = self.is_valid_model(node.transactions)
        if not accuracy_check:
            print(f"Node {node.idx} failed the model accuracy check. Node discarded.")
            return None

        # Simulate validator votes for the block
        votes = self.node_votes(node)
        print(f"Validator Votes: {votes}")

        if len(votes) >= self.consensus_threshold:
            self.chain.append(node)  
            self.c_node_idx += 1
            self.transactions = []  # Clear transactions 
            
            print(f"Node {node.idx} successfully added to the blockchain.")
        else:          
            print(f"Node {node.idx} failed to reach consensus. Node discarded.")  
        return node

    
    # Simulate validators voting on the block   
    def node_votes(self, node):
        votes = []
        
        for validator in self.validators:
            if self.data_integrity(node.transactions) and self.is_valid_model(node.transactions):
                votes.append(validator)
                
        return votes

        
    def data_integrity(self, transactions):
        """
        Checks for missing values, duplicates, and incorrect data in transactions.
        """
        # Check for missing values, duplicates, and incorrect data
        for transaction in transactions:
            # Assuming 'model data' is a numeric value, or convert it into a pandas dataframe if needed
            if isinstance(transaction['model data'], pd.DataFrame):
                missing_values = transaction['model data'].isnull().sum().sum()
                duplicate_values = transaction['model data'].duplicated().sum()
                incorrect_data = 0
                # Check for incorrect data types (for example, if expected to be numeric)
                for col in transaction['model data'].select_dtypes(include=[np.number]).columns:
                    if not pd.api.types.is_numeric_dtype(transaction['model data'][col]):
                        incorrect_data += 1
                
                # Check if there are any issues
                if missing_values > 0 or duplicate_values > 0 or incorrect_data > 0:
                    return False
            else:
                # Assuming model data is numeric
                if not isinstance(transaction['model data'], (int, float)):
                    return False
        
        return True
    
    def is_valid_model(self, transactions):
        """
        Checks if the model accuracy is above the expected threshold.
        """
        expected_m_accuracy = 80 
        
        for transaction in transactions:
            if 'model accuracy' in transaction:
                m_accuracy = transaction['model accuracy']
                # Ensure accuracy is above the threshold
                if m_accuracy < expected_m_accuracy:
                    return False
        
        return True  

    def get_last_node(self):
        return self.chain[-1]

    
    def display_chain(self):
        for node in self.chain:
            print(f"##### Session Starts ####")
            print(f"Node {node.idx}: {node.hash} | Validator: {node.validator} | Proof: {node.proof}")
            print(f"----------------------------")

# Example usage
blockchain = Blockchain_Network()
blockchain.transaction_session('AI_Vendor_X', 'Edge_Device_1', 100, 85)  # Add a transaction with model data and accuracy
blockchain.mine_node()  # Mine the node and check content matching
blockchain.display_chain()  # Display the blockchain
blockchain.transaction_session('AI_Vendor_Y', 'Edge_Device_2', 150, 90)
blockchain.mine_node()  # Mine the node and check content matching
blockchain.display_chain()  # Display the blockchain
blockchain.transaction_session('AI_Vendor_Y', 'Edge_Device_2', 150, 70)  # Add another transaction with model data and accuracy


blockchain.mine_node()  # Mine the node and check content matching
blockchain.display_chain()  # Display the blockchain


In [ ]:
import hashlib
import time
import random as rand

# node representing each block
class Node:
    def __init__(self, idx, prev_hash, t_stamp, transactions, proof, validator):
        self.idx = idx
        self.prev_hash = prev_hash
        self.t_stamp = t_stamp
        self.transactions = transactions
        self.proof = proof
        self.validator = validator
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        node_str = f"{self.idx}{self.prev_hash}{self.t_stamp}{self.transactions}{self.proof}{self.validator}"
        return hashlib.sha256(node_str.encode('utf-8')).hexdigest()
        
class Blockchain_Network:
    
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.c_node_idx = 0         
        self.validators = ['R_Edge_Server_A', 'R_Edge_Server_B', 'R_Edge_Server_C', 'R_Edge_Server_D']
        self.add_node()
        self.consensus_threshold = 2  

    def add_node(self):
        # initial block
        if self.c_node_idx == 0:
            start_node = Node(0, "0", time.time(), [], 'Start', 'System')
            self.chain.append(start_node)
            self.c_node_idx += 1

    # simulate a transaction creation  data: model's data, accuracy: model's accuracy
    def transaction_session(self, source, receiver, data, accuracy):
        transaction = {"AI Vendor": source, "Edge Device": receiver, "model data": data, "model accuracy": accuracy}
        self.transactions.append(transaction)

        
    # Simulate the mining process
    def mine_node(self):
        
        validator = rand.choice(self.validators) 
        t_stamp = time.time()
        
        # Simulate proof of authority by generating a random proof
        proof = rand.randint(150, 8500)  
        node = Node(self.c_node_idx, self.chain[-1].hash, t_stamp, self.transactions, proof, validator)

        # Check for data integrity and accuracy before adding the node
        integrity_check = self.data_integrity(node.transactions)
        if not integrity_check:
            print(f"Node {node.idx} failed the data integrity check. Node discarded.")
            return None

        accuracy_check = self.is_valid_model(node.transactions)
        if not accuracy_check:
            print(f"Node {node.idx} failed the model accuracy check. Node discarded.")
            return None

        # Simulate validator votes for the block
        votes = self.node_votes(node)
        print(f"Validator Votes: {votes}")

        if len(votes) >= self.consensus_threshold and (integrity_check and accuracy_check is True):
            self.chain.append(node)  
            self.c_node_idx += 1
            self.transactions = [] 
            
            print(f"Node {node.idx} successfully added to the blockchain.")
        else:          
            print(f"Node {node.idx} failed to reach consensus. Node discarded.")  
        return node

    
     # Simulate validators voting on the block   
    def node_votes(self, node):
        votes = []
        
        for validator in self.validators:
            if self.data_integrity(node.transactions) and self.is_valid_model(node.transactions):
                votes.append(validator)
            # voters = rand.choice([True, False])
            # if voters and  self.data_integrity(node.transactions) and self.is_valid_model(node.transactions) is True:
            #         votes.append(validator)
                
            # if self.data_integrity(node.transactions):
            #     if self.is_valid_model(node.transactions):
                    # votes.append(validator)
            
            # if rand.choice([True, False]):
            #     votes.append(validator)     
        
        return votes

        
    def data_integrity(self, transactions):
        
        # accuracy, consistency, data mismatch, and outliers 
        for transaction in transactions:
   
            if 'model data' in transaction:
                data = transaction['model data']
                missing_values = data.isnull().sum().sum()
                duplicate_values = data.duplicated().sum()
                incorrect_data = 0
                # outliers = 0
                
                for col in data.select_dtypes(include=[np.number]).columns:
                    if not pd.api.types.is_numeric_dtype(data[col]):
                        incorrect_data += 1
                        
                # for col in data.select_dtypes(include=[np.number]).columns:
                #     mean = data[col].mean()
                #     std_dev = data[col].std()
                #     outliers_in_col = data[(data[col] < mean - 3*std_dev) | (data[col] > mean + 3 *std_dev)]
                #     outliers += outliers_in_col.shape[0] 
                    
                # print(f"Integrity check results - Missing data: {missing_values}, Duplicates: {duplicate_values}, Incorrect data: {incorrect_data}, Outliers: {outliers}")
                # print(f"Integrity check results - Missing data: {missing_values}, Duplicates: {duplicate_values}, Incorrect data: {incorrect_data}")
                # if missing_values > 0 or duplicate_values > 0 or incorrect_data > 0 or outliers > 0:
                if missing_values > 0 or duplicate_values > 0 or incorrect_data > 0:
                    return False
                return True
            
        
    def is_valid_model(self, transactions):
        expected_m_accuracy = 80 
       
        for transaction in transactions:
            if 'accuracy' in transaction:
                m_accuracy = transaction['accuracy']
                m_accuracy >=  expected_m_accuracy 
                return False
            return True  


    def get_last_node(self):
        return self.chain[-1]

    
    def display_chain(self):
        for node in self.chain:
            print(f"##### Session Starts ####")
            print(f"Node {node.idx}: {node.hash} | Validator: {node.validator} | Proof: {node.proof}")
            # print(f"Transactions: {node.transactions}")
            # print(f"##### Session Ends ####")
            print(f"----------------------------")
            # print(f"---Updated block chain------")
            # print(f"############################")


In [ ]:

        #  
        # missing_values = data.isnull().sum().sum()
        # duplicate_values = data.duplicated().sum()
        
        # incorrect_data = 0
        # outliers = 0
        
        # for col in data.select_dtypes(include=[np.number]).columns:
        #     if not pd.api.types.is_numeric_dtype(data[col]):
        #         incorrect_data += 1
                
        # for col in data.select_dtypes(include=[np.number]).columns:
        #     mean = data[col].mean()
        #     std_dev = data[col].std()
        #     outliers_in_col = data[(data[col] < mean - 3*std_dev) | (data[col] > mean + 3*std_dev)]
        #     outliers += outliers_in_col.shape[0] 
       
        # print(f"Integrity check results - Missing data: {missing_values}, Duplicates: {duplicate_values}, Incorrect data: {incorrect_data}, Outliers: {outliers}")
        
        # if missing_values > 0 or duplicate_values > 0 or incorrect_data > 0 or outliers > 0:
        #     return False
        # return True

In [ ]:
def data_integrity(data):
    
    # accuracy, consistency, data mismatch, and outliers 
    missing_values = data.isnull().sum().sum()
    duplicate_values = data.duplicated().sum()
    
    incorrect_data = 0
    outliers = 0
    
    for col in data.select_dtypes(include=[np.number]).columns:
        if not pd.api.types.is_numeric_dtype(data[col]):
            incorrect_data += 1

    for col in data.select_dtypes(include=[np.number]).columns:
            mean = data[col].mean()
            std_dev = data[col].std()
            outliers_in_col = data[(data[col] < mean - 3*std_dev) | (data[col] > mean + 3*std_dev)]
            outliers += outliers_in_col.shape[0] 
        
    # Print out the results
    print(f"Missing Values: {missing_values}")
    print(f"Duplicate Rows: {duplicate_values}")
    print(f"Incorrect Data Type Columns: {incorrect_data}")
    print(f"Outlier Values: {outliers}")

    return missing_values, duplicate_values, incorrect_data, outliers

# Example usage:
# Assuming you have a pandas DataFrame `df`
# df = pd.read_csv('your_data.csv')
# missing_values, duplicate_values = check_data_integrity(df)

data = Occupancy

missing_values, duplicate_values, incorrect_data, outliers = data_integrity(data)

In [ ]:
powerConsumption

In [ ]:
Occupancy

In [ ]:
# datasets
activity_wsdata = pd.read_csv('C:/Users/moyin/Downloads/data/activity data.csv')

target = 'activity'
X = activity_wsdata.drop(columns=[target])
y = activity_wsdata[target]

# split data
WS_train, WS_test, ws_train, ws_test = train_test_split(X, y, test_size=0.5, random_state=42)
data_train, data_test, goal_train, goal_test = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
e_Device_X, e_Data_X, e_Device_Y, e_Data_Y = train_test_split(model_train, target_train, train_size=0.5, random_state=42)
e_Server1_X, e_Server2_X, e_Server1_Y, e_Server2_Y = train_test_split(edgeDevice_X, edgeData_X, train_size=0.5, random_state=42)


In [ ]:
# activity_wsdata['activity'].unique()

In [ ]:
print(len(X_train))
print(len(model_train))
print(len(edgeDevice_X))
print(len(edgeServer1_X))


In [ ]:
import hashlib
import time
import random as rand

# node representing each block
class Node:
    def __init__(self, idx, prev_hash, t_stamp, transactions, proof, validator):
        self.idx = idx
        self.prev_hash = prev_hash
        self.t_stamp = t_stamp
        self.transactions = transactions
        self.proof = proof
        self.validator = validator
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        node_str = f"{self.idx}{self.prev_hash}{self.t_stamp}{self.transactions}{self.proof}{self.validator}"
        return hashlib.sha256(node_str.encode('utf-8')).hexdigest()
        
class Blockchain_Network:
    
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.c_node_idx = 0         
        self.validators = ['R_Edge_Server_A', 'R_Edge_Server_B', 'R_Edge_Server_C', 'R_Edge_Server_D']
        self.add_node()
        # minimum number of validators required for the consensus
        self.consensus_threshold = 2

    def add_node(self):
        # initial block
        if self.c_node_idx == 0:
            start_node = Node(0, "0", time.time(), [], 'Start', 'System')
            self.chain.append(start_node)
            self.c_node_idx += 1

    # simulate a transaction creation - # amt: data size of the AI model parameter
    def transaction_session(self, source, receiver, data):
        transaction = {"AI Vendor": source, "Edge Device": receiver, "model data": data}
        self.transactions.append(transaction)
        
    # Simulate the mining process
    def mine_node(self):
        
        validator = rand.choice(self.validators) 
        t_stamp = time.time()
        
        # Simulate proof of authority by generating a random proof
        proof = rand.randint(150, 850000)  
        node = Node(self.c_node_idx, self.chain[-1].hash, t_stamp, self.transactions, proof, validator)
        
        # Simulate validator votes for the block
        votes = self.node_votes(node)
        print(votes)

        if len(votes) >= self.consensus_threshold:
            self.chain.append(node)  
            self.c_node_idx += 1
            self.transactions = [] 
            
            print(f"Node {node.idx} successfully added to the blockchain.")
            
        else:
            
            print(f"Node {node.idx} failed to reach consensus. Block discarded.")
        
        # self.chain.append(node)
        # self.c_node_idx += 1
        # self.transactions = []  
        return node

    def node_votes(self, node):
        # Simulate validators voting on the block. Each validator votes based on the block's validator.
        votes = []
        
        for validator in self.validators:
            # Validators check if the node's content match 
            if self.is_valid_mparameter(node.transactions):
                
                votes.append(validator)
        
        return votes

        # works for making the decision using randomization
        # for validator in self.validators:
        #     # Validators vote if they are not the one who created the block
        #     if rand.choice([True, False]):
                
        #         votes.append(validator)
                
        # return votes
        
    def is_valid_mparameter(self, transactions):

        expected_model_accuracy = 10 # confirm the value is percentage or decimal
        
        # Use model accuracy in this variable. Cons: it uses numeric validation (verification is solely based on the model's accuracy figure, not the content).
        # Will work best when a single model is sent to the edge servers for verification
        
        for transaction in transactions:
            if transaction['data'] != expected_model_accuracy: # either use if not or use a lowerbound figure to eliminate the models.
                return False  
        return True  
    
    def get_last_node(self):
        return self.chain[-1]

    
    def display_chain(self):
        for node in self.chain:
            # print(f"##### Session Starts ####")
            print(f"Node {node.idx}: {node.hash} | Validator: {node.validator} | Proof: {node.proof}")
            # print(f"Transactions: {node.transactions}")
            print(f"##### Session Ends ####")
            print(f"----------------------------")
            # print(f"---Updated block chain------")
            # print(f"############################")


In [ ]:
# Initialize the blockchain
PoA_consensus = Blockchain_Network()

# Simulate creating transactions
    # source: AI Marketplace represented as App_VendorID
    # receiver: edge devices defined as ED_RegionID_DeviceID where ED represents the edge device

# Display the updated blockchain
PoA_consensus.display_chain()

# convert the data to a single array representing the dataset...
PoA_consensus.transaction_session("App_V01", "ED_R1_AR1", model_train)
PoA_consensus.transaction_session("App_V01", "ED_R2_AR2", WS_train)

# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

PoA_consensus.transaction_session("App_V03", "ED_R4_AR1", edgeDevice_X)
PoA_consensus.transaction_session("App_V04", "ED_R3_AR2", data_train)

# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

PoA_consensus.transaction_session("App_V02", "ED_R1_AR2", e_Device_X)
PoA_consensus.transaction_session("App_V02", "ED_R1_AR4", edgeServer1_X)

# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

PoA_consensus.transaction_session("App_V05", "ED_R2_AR3", e_Server1_X)
PoA_consensus.transaction_session("App_V01", "ED_R1_AR5", edgeServer2_X)

# Simulate mining a block
mined_node = PoA_consensus.mine_node()

# # Display the blockchain
# PoA_consensus.display_chain()

PoA_consensus.transaction_session("App_V02", "ED_R4_AR1", edgeDevice_X)
PoA_consensus.transaction_session("App_V05", "ED_R1_AR2", WS_train)

# # Mine another block
# PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

PoA_consensus.transaction_session("App_V01", "ED_R2_AR1", e_Server1_X)
PoA_consensus.transaction_session("App_V02", "ED_R4_AR4", model_train)

# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

PoA_consensus.transaction_session("App_V04", "ED_R4_AR1", edgeDevice_X)
PoA_consensus.transaction_session("App_V05", "ED_R2_AR3", X_train)

# Mine another block
PoA_consensus.mine_node()

# # Display the updated blockchain
# PoA_consensus.display_chain()

PoA_consensus.transaction_session("App_V03", "ED_R3_AR5", e_Data_X)
PoA_consensus.transaction_session("App_V01", "ED_R1_AR2", edgeData_X)

# Mine another block
PoA_consensus.mine_node()

# Display the updated blockchain
PoA_consensus.display_chain()


In [ ]:
# things to note
# the voting is randomized, so the key decision factor used is the consensus threshold
# the actual data being passed to it at this point, isn't being reflected. because it is actually not checking the data.
## the validator is deciding based on the randomized number of votes present at a given time.

In [ ]:
import pandas as pd

def validate_dataset(df):
    """
    Validates the given dataset (Pandas DataFrame) for the following conditions:
    - No missing values in required columns
    - Correct data types
    - Age should be an integer within a valid range
    - Name should not be empty and should consist of alphabets

    Args:
    df (pd.DataFrame): The dataset to validate

    Returns:
    dict: A dictionary with validation results
    """
    validation_results = {
        "missing_values": {},
        "invalid_data_types": {},
        "invalid_ages": [],
        "invalid_names": []
    }

    # Check for missing values in specific columns
    required_columns = ['id', 'name', 'age']
    for col in required_columns:
        missing = df[col].isnull().sum()
        if missing > 0:
            validation_results['missing_values'][col] = missing

    # Check data types
    if not pd.api.types.is_integer_dtype(df['age']):
        validation_results['invalid_data_types']['age'] = 'Expected integer type for age'

    if not pd.api.types.is_string_dtype(df['name']):
        validation_results['invalid_data_types']['name'] = 'Expected string type for name'

    # Check for invalid age values (e.g., not an integer, or out of range)
    for index, row in df.iterrows():
        if not (18 <= row['age'] <= 100):
            validation_results['invalid_ages'].append((index, row['age']))

    # Check for invalid name values (e.g., empty string or non-alphabet characters)
    for index, row in df.iterrows():
        if not row['name'].isalpha():
            validation_results['invalid_names'].append((index, row['name']))

    return validation_results

# Example usage:
data = {
    'id': [1, 2, 3, 4],
    'name': ['Alice', 'Bob', '1234', 'Charlie'],
    'age': [25, 32, 101, None]
}

df = pd.DataFrame(data)

validation_result = validate_dataset(df)
print(validation_result)


In [ ]:
import hashlib
import time
import random
import pandas as pd

class Block:
    def __init__(self, index, previous_hash, timestamp, transactions, proof, validator):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.transactions = transactions
        self.proof = proof
        self.validator = validator
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = f"{self.index}{self.previous_hash}{self.timestamp}{self.transactions}{self.proof}{self.validator}"
        return hashlib.sha256(block_string.encode('utf-8')).hexdigest()

class Blockchain:
    def __init__(self, df):
        self.chain = []
        self.transactions = []
        self.current_block_index = 0
        self.validators = ["Validator1", "Validator2", "Validator3"]  # List of validators
        self.df = df  # DataFrame with valid transaction data (including amount and content)
        self.add_block()
        self.consensus_threshold = 2  # Minimum number of validators to approve a block

    def add_block(self):
        # Add the genesis block
        if self.current_block_index == 0:
            genesis_block = Block(0, "0", time.time(), [], "genesis", "System")
            self.chain.append(genesis_block)
            self.current_block_index += 1

    def create_transaction(self, sender, recipient, amount, content):
        # Simulate creating a transaction
        transaction = {"sender": sender, "recipient": recipient, "amount": amount, "content": content}
        self.transactions.append(transaction)

    def mine_block(self):
        # Simulate the mining process with consensus based on matching amounts and content
        validator = random.choice(self.validators)  # Randomly select a validator
        timestamp = time.time()
        proof = random.randint(0, 1000000)  # Simulate proof of authority by generating a random proof
        block = Block(self.current_block_index, self.chain[-1].hash, timestamp, self.transactions, proof, validator)
        
        # Simulate validator votes for the block based on matching amounts and content
        votes = self.get_votes_for_block(block)

        if len(votes) >= self.consensus_threshold:
            self.chain.append(block)  # Add block to chain if it reaches consensus
            self.current_block_index += 1
            self.transactions = []  # Reset transactions after mining
            print(f"Block {block.index} successfully added to the blockchain.")
        else:
            print(f"Block {block.index} failed to reach consensus. Block discarded.")
        
        return block

    def get_votes_for_block(self, block):
        # Simulate validators voting on the block based on matching amounts and content
        votes = []
        for validator in self.validators:
            # Validators check if the block's transactions match expected amounts and content in the DataFrame
            if self.is_valid_transaction(block.transactions):
                votes.append(validator)
        return votes

    def is_valid_transaction(self, transactions):
        # In this scenario, we'll define valid transactions as those that match both the amount and content in the DataFrame
        for transaction in transactions:
            amount_match = self.df[self.df['amount'] == transaction['amount']]
            content_match = self.df[(self.df['amount'] == transaction['amount']) & 
                                    (self.df['content'] == transaction['content'])]
            if amount_match.empty or content_match.empty:
                return False  # Reject the block if any transaction doesn't match the DataFrame criteria
        return True  # Accept the block if all transactions match the expected amount and content

    def get_last_block(self):
        return self.chain[-1]

    def display_chain(self):
        for block in self.chain:
            print(f"Block {block.index}: {block.hash} | Validator: {block.validator} | Proof: {block.proof}")
            print(f"Transactions: {block.transactions}")
            print(f"----------------------------")

# Sample DataFrame with valid transaction data (amounts and contents)
data = {
    'sender': ['Alice', 'Bob', 'Charlie'],
    'recipient': ['Bob', 'Charlie', 'Alice'],
    'amount': [10, 10, 10],  # Expected amounts
    'content': ['Payment for service', 'Payment for goods', 'Payment for service']  # Expected content
}
df = pd.DataFrame(data)

# Test the blockchain with consensus based on matching amounts and content
poa_blockchain = Blockchain(df)

# Simulate creating transactions
poa_blockchain.create_transaction("Alice", "Bob", 10, "Payment for service")
poa_blockchain.create_transaction("Bob", "Charlie", 10, "Payment for goods")

# Simulate mining a block with consensus
poa_blockchain.mine_block()

# Display the blockchain after mining
poa_blockchain.display_chain()

# Simulate more transactions and mining
poa_blockchain.create_transaction("Charlie", "Alice", 10, "Payment for service")
poa_blockchain.create_transaction("Alice", "Bob", 10, "Payment for service")

# Simulate mining another block with consensus
poa_blockchain.mine_block()

# Display the updated blockchain
poa_blockchain.display_chain()


In [ ]:

# # Step 4: Initialize the model (Logistic Regression)
# model = LogisticRegression(max_iter=200)

# # Step 5: Train the model using the training data
# model.fit(X_train, y_train)

# # Step 6: Make predictions on the test set
# y_pred = model.predict(X_test)

# # Step 7: Evaluate the model (Accuracy)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Model Accuracy: {accuracy * 100:.2f}%")


In [ ]:
def displayInfo (training, edge1, edge2, edge3, edge4):
    print ("Precision on Training Data: ", training)
    print ("Precision on edge1 Data: ", edge1)
    print ("Precision on edge2 Data: ", edge2)
    print ("Precision on edge3 Data: ", edge3)
    print ("Precision on edge4 Data: ", edge4)

def calculateAccuracy (Y_predicted, Y_true, datasetType: str, model):
    if datasetType == "Categorical":
        return metrics.precision_score(Y_true, Y_predicted,average='macro')
    return metrics.mean_squared_error(Y_true, Y_predicted)


def runModel(model, X_test, Y_test,datasetType: str): 
    Y_predicted = model.predict(X_test)
    return calculateAccuracy (Y_predicted, Y_test,datasetType, model)

In [ ]:
# models
def trainCategoricalModel (X,Y):
    return LogisticRegression().fit(X, Y)

def trainCatergoricalModel_ (X,Y):
    return GaussianNB().fit(X, Y)

def trainCatModel (X,Y):
    model  = svm.SVC(kernel='linear')
    return model.fit(X, Y)

def trainContinuousModel (X,Y):
    return LinearRegression().fit(X, Y)

def trainContinuousModel_ (X,Y):
    Y = Y.astype('int')
    model = KNeighborsClassifier(n_neighbors=1)
    return model.fit(X,Y)

def trainConModel(X,Y):
    Y = Y.astype('int')
    model = RandomForestRegressor(n_estimators= 1000, random_state=42)
    return model.fit(X,Y)

In [ ]:
def loadData (df,predictName: str,datasetType: str):

    X = df.drop(columns=[predictName])  
    y = df[predictName]                
    
    # First split: Split data into two halves (50% training and 50% for further splits)
    model_train_X, X_remaining, model_train_Y, y_remaining = train_test_split(X, y, train_size=0.5, random_state=42)
    
    # Second split: Split the remaining data into 2 (50% for edge server 1 and 50% for edge server 2)
    edgeServer1_X, edgeServer2_X, edgeServer1_Y, edgeServer2_Y = train_test_split(X_remaining, y_remaining, train_size=0.5, random_state=42)
    
    # Third split: Split the remaining data into 2 (50% for edge server 3 and 50% for edge server 4)
    edgeServer3_X, edgeServer4_X, edgeServer3_Y, edgeServer4_Y = train_test_split(X_remaining, y_remaining, train_size=0.5, random_state=42)
    
    if datasetType == "Categorical":
        model = trainCategoricalModel (model_train_X, model_train_Y)
        
    else:
        model = trainContinuousModel (model_train_X, model_train_Y)
        
        
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = runModel (model, model_train_X, model_train_Y,datasetType), runModel(model,edgeServer1_X, edgeServer1_Y,datasetType), runModel(model,edgeServer2_X, edgeServer2_Y,datasetType), runModel(model,edgeServer3_X, edgeServer3_Y,datasetType), runModel(model,edgeServer4_X, edgeServer4_Y,datasetType)

    return trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy


In [ ]:
def loaddata (df,predictName: str,datasetType: str):
    
    X = df.drop(columns=[predictName])  
    y = df[predictName]                
    
    # First split: Split data into two halves (50% training and 50% for further splits)
    model_train_X, X_remaining, model_train_Y, y_remaining = train_test_split(X, y, train_size=0.5, random_state=42)
    
    # Second split: Split the remaining data into 2 (50% for edge server 1 and 50% for edge server 2)
    edgeServer1_X, edgeServer2_X, edgeServer1_Y, edgeServer2_Y = train_test_split(X_remaining, y_remaining, train_size=0.5, random_state=42)
    
    # Third split: Split the remaining data into 2 (50% for edge server 3 and 50% for edge server 4)
    edgeServer3_X, edgeServer4_X, edgeServer3_Y, edgeServer4_Y = train_test_split(X_remaining, y_remaining, train_size=0.5, random_state=42)
    
    if datasetType == "Categorical":
        
        model_a = trainCatergoricalModel_(model_train_X, model_train_Y)
        
    else:
        
        model_a = trainContinuousModel_(model_train_X, model_train_Y)   
    

    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = runModel (model_a, model_train_X, model_train_Y,datasetType), runModel(model_a,edgeServer1_X, edgeServer1_Y,datasetType), runModel(model_a,edgeServer2_X, edgeServer2_Y,datasetType), runModel(model_a,edgeServer3_X, edgeServer3_Y,datasetType), runModel(model_a,edgeServer4_X, edgeServer4_Y,datasetType)
    
    return training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A
    

In [ ]:
def load_data (df,predictName: str,datasetType: str):
    
    X = df.drop(columns=[predictName])  
    y = df[predictName]                
    
    # First split: Split data into two halves (50% training and 50% for further splits)
    model_train_X, X_remaining, model_train_Y, y_remaining = train_test_split(X, y, train_size=0.5, random_state=42)
    
    # Second split: Split the remaining data into 2 (50% for edge server 1 and 50% for edge server 2)
    edgeServer1_X, edgeServer2_X, edgeServer1_Y, edgeServer2_Y = train_test_split(X_remaining, y_remaining, train_size=0.5, random_state=42)
    
    # Third split: Split the remaining data into 2 (50% for edge server 3 and 50% for edge server 4)
    edgeServer3_X, edgeServer4_X, edgeServer3_Y, edgeServer4_Y = train_test_split(X_remaining, y_remaining, train_size=0.5, random_state=42)

    
    if datasetType == "Categorical":
        
        model_b = trainCatModel(model_train_X, model_train_Y)
        
    else:
       
        model_b = trainConModel(model_train_X, model_train_Y)
        
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = runModel (model_b, model_train_X, model_train_Y,datasetType), runModel(model_b,edgeServer1_X, edgeServer1_Y,datasetType), runModel(model_b,edgeServer2_X, edgeServer2_Y,datasetType), runModel(model_b,edgeServer3_X, edgeServer3_Y,datasetType), runModel(model_b,edgeServer4_X, edgeServer4_Y,datasetType) 
    
        
    return training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B


In [ ]:
# dataset for each case
def Occupancy():
    print("-------------- Occupancy Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/Occupancy.csv')
    df.drop(columns = ["date"],inplace = True)
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'Occupancy',"Categorical")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = loaddata (df,'Occupancy',"Categorical")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'Occupancy',"Categorical")
    return 


def powerConsumption():
    print("-------------- Power Consumption Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/Tetuan City power consumption.csv')
    df.drop(columns = ['DateTime','Zone 2  Power Consumption', 'Zone 3  Power Consumption'],inplace = True)
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'Zone 1 Power Consumption',"Continuous")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = loaddata (df,'Zone 1 Power Consumption',"Continuous")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'Zone 1 Power Consumption',"Continuous")
    return 

def activity_wsdata():
    print("-------------- Wearable Sensor Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/activity data.csv')
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'activity',"Categorical")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = loaddata (df,'activity',"Categorical")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'activity',"Categorical")
    return 

def ALEdata():
    print("-------------- ALE Sensor Data Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/ALE in Sensor.csv')
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'sd_ale',"Continuous")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A  = loaddata (df,'sd_ale',"Continuous")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'sd_ale',"Continuous")
    return 

In [ ]:
print("-------------- ALE Sensor Data Dataset Info -------------------- ")
df = pd.read_csv('C:/Users/moyin/Downloads/data/ALE in Sensor.csv')
trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'sd_ale',"Continuous")
training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A  = loaddata (df,'sd_ale',"Continuous")
training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'sd_ale',"Continuous")

In [ ]:
def Occupancy():
    print("-------------- Occupancy Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/Occupancy.csv')
    df.drop(columns = ["date"],inplace = True)
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'Occupancy',"Categorical")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = loaddata (df,'Occupancy',"Categorical")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'Occupancy',"Categorical")
    displayInfo(trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy)
    print("------------------------------------------------------------")
    displayInfo(training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A)
    print("------------------------------------------------------------")
    displayInfo(training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B)
    return 

In [ ]:
def powerConsumption():
    print("-------------- Power Consumption Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/Tetuan City power consumption.csv')
    df.drop(columns = ['DateTime','Zone 2  Power Consumption', 'Zone 3  Power Consumption'],inplace = True)
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'Zone 1 Power Consumption',"Continuous")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = loaddata (df,'Zone 1 Power Consumption',"Continuous")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'Zone 1 Power Consumption',"Continuous")
    displayInfo(trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy)
    print("------------------------------------------------------------")
    displayInfo(training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A)
    print("------------------------------------------------------------")
    displayInfo(training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B)
    return 

In [ ]:
def accelerometer_w():
    print("-------------- Accelerometer Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/accelerometer.csv')
#     df.drop(columns = ['wconfid','pctid', 'x', 'y', 'z'],inplace = True)
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'wconfid',"Continuous")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = loaddata (df,'wconfid',"Continuous")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'wconfid',"Continuous")
    displayInfo(trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy)
    print("------------------------------------------------------------")
    displayInfo(training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A)
    print("------------------------------------------------------------")
    displayInfo(training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B)
    return 

In [ ]:
def activity_wsdata():
    print("-------------- Wearable Sensor Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/activity data.csv')
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'activity',"Categorical")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = loaddata (df,'activity',"Categorical")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'activity',"Categorical")
    displayInfo(trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy)
    print("------------------------------------------------------------")
    displayInfo(training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A)
    print("------------------------------------------------------------")
    displayInfo(training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B)
    return 

In [ ]:
def ALEdata():
    print("-------------- ALE Sensor Data Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/ALE in Sensor.csv')
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'sd_ale',"Continuous")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A  = loaddata (df,'sd_ale',"Continuous")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'sd_ale',"Continuous")
    displayInfo(trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy)
    print("------------------------------------------------------------")
    displayInfo(training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A)
    print("------------------------------------------------------------")
    displayInfo(training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B)
    return 

In [ ]:
def banknote_data():
    print("-------------- Bank Note Authentication Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/banknote_authen.txt')
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'class',"Categorical")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A = loaddata (df,'class',"Categorical")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'class',"Categorical")
    displayInfo(trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy)
    print("------------------------------------------------------------")
    displayInfo(training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A)
    print("------------------------------------------------------------")
    displayInfo(training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B)
    return

In [ ]:
def RSSI():
    print("-------------- BLE RSSI Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/BLE RSSI.csv')
    df.drop(columns = ['name'],inplace = True)
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'locationStatus', "Categorical")
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A  = loaddata (df,'locationStatus', "Categorical")
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B  = load_data (df,'locationStatus', "Categorical")
    displayInfo(trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy)
    print("------------------------------------------------------------")
    displayInfo(training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A)
    print("------------------------------------------------------------")
    displayInfo(training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B)
    return 

In [ ]:
def e_grid():
    print("-------------- Simulated Electrical Grid Dataset Info -------------------- ")
    df = pd.read_csv('C:/Users/moyin/Downloads/data/UCI_named.csv')
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'stabf', 'Categorical')
    training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A  = loaddata (df,'stabf', 'Categorical')
    training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B = load_data (df,'stabf', 'Categorical')
    displayInfo(trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy)
    print("------------------------------------------------------------")
    displayInfo(training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A)
    print("------------------------------------------------------------")
    displayInfo(training_B, edgeServer1_B, edgeServer2_B, edgeServer3_B, edgeServer4_B)
    return  

In [ ]:
# if __name__=="__main__":
#     Occupancy()
#     powerConsumption()
#     accelerometer_w()
#     activity_wsdata()
#     ALEdata()
#     banknote_data()
#     RSSI()
#     e_grid()

In [ ]:
Occupancy()

In [ ]:
powerConsumption()

In [ ]:
accelerometer_w()

In [ ]:
activity_wsdata()

In [ ]:
ALEdata()

In [ ]:
banknote_data()

In [ ]:
RSSI()

In [ ]:
e_grid()

In [ ]:
def loadData (df,predictName: str,datasetType: str):

    X = df.drop(columns=[predictName])  
    y = df[predictName]                
    
    # First split: Split data into two halves (50% training and 50% for further splits)
    model_train_X, X_remaining, model_train_Y, y_remaining = train_test_split(X, y, train_size=0.5, random_state=42)
    
    # Second split: Split the remaining data into 2 (50% for edge server 1 and 50% for edge server 2)
    edgeServer1_X, edgeServer2_X, edgeServer1_Y, edgeServer2_Y = train_test_split(X_remaining, y_remaining, train_size=0.5, random_state=42)
    
    # Third split: Split the remaining data into 2 (50% for edge server 3 and 50% for edge server 4)
    edgeServer3_X, edgeServer4_X, edgeServer3_Y, edgeServer4_Y = train_test_split(X_remaining, y_remaining, train_size=0.5, random_state=42)
    
    if datasetType == "Categorical":
        model = trainCategoricalModel (model_train_X, model_train_Y)
        
    else:
        model = trainContinuousModel (model_train_X, model_train_Y)
        
        
    trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = runModel (model, model_train_X, model_train_Y,datasetType), runModel(model,edgeServer1_X, edgeServer1_Y,datasetType), runModel(model,edgeServer2_X, edgeServer2_Y,datasetType), runModel(model,edgeServer3_X, edgeServer3_Y,datasetType), runModel(model,edgeServer4_X, edgeServer4_Y,datasetType)

    return trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy



In [ ]:
print("-------------- Occupancy Dataset Info -------------------- ")

df = pd.read_csv('C:/Users/moyin/Downloads/data/Occupancy.csv')
df.drop(columns = ["date"],inplace = True)
trainingAccuacy, edgeServer1Accuracy, edgeServer2Accuracy, edgeServer3Accuracy, edgeServer4Accuracy = loadData(df,'Occupancy',"Categorical")

In [ ]:

training_A, edgeServer1_A, edgeServer2_A, edgeServer3_A, edgeServer4_A  = loaddata (df,'locationStatus', "Categorical")